# 順列と組みあわせ

## 重複を許す組み合わせ
結晶格子の座標をプログラムで作りだす場合を考えてみましょう。2次元の10x10の単純正方格子の座標は、2重ループで簡単に書けます。

In [ ]:
pos = []
for x in range(10):
    for y in range(10):
        pos.append((x,y))
print(pos)

同じように、4x4x4の3次元の立方格子の座標は3重ループで書けます。

In [ ]:
pos = []
for x in range(4):
    for y in range(4):
        for z in range(4):
            pos.append((x,y,z))
print(pos)

関数にしてみましょう。

In [ ]:
def combination3(a,b,c):
    pos = []
    for x in range(a):
        for y in range(b):
            for z in range(c):
                pos.append((x,y,z))
    return pos

print(combinations(4,4,4))

イテレータにすると、使いやすくなります。

In [ ]:
def combination3(a,b,c):
    for x in range(a):
        for y in range(b):
            for z in range(c):
                yield x,y,z

for pos in combination3(4,4,4):
    print(pos)

数値の範囲を個別に渡す代わりに、リストでまとめて渡します。

In [ ]:
def combination3(L):
    for x in range(L[0]):
        for y in range(L[1]):
            for z in range(L[2]):
                yield x,y,z

for pos in combination3((4,4,4)):
    print(pos)

数字の組みあわせでなく、リストや集合の要素を組みあわせられるようにしましょう。

In [ ]:
def combination3(L):
    for x in L[0]:
        for y in L[1]:
            for z in L[2]:
                yield x,y,z

for pos in combination3(((1,2,3),(4,5,6),(7,8,9))):
    print(pos)

再帰を使えば、n次元に一般化できます。再帰とは、関数のなかから、その関数自身を呼びだすことです。再帰する場合には、関数の中で終了条件(再帰をやめる条件)を必ず書いておく必要があります。

In [ ]:
def combinations(L):
    if len(L) == 0: # terminator
        yield []
    else:
        for x in L[0]:
            for y in combinations(L[1:]):
                yield [x]+y

for pos in combinations([(0,1),(0,2),(0,4),(0,8)]):
    print(pos)

## 重複なしの組み合わせ
1〜10のなかから異なる2つを選んだ組みあわせを作りたい場合は、条件分けします。

In [ ]:
s = []
for i in range(1,11):
    for j in range(1,11):
        if i < j:
            s.append((i,j))
print(s)

でも、jの繰り返し範囲を調節すれば、if文も要らなくなります。

In [ ]:
s = []
for i in range(1,11):
    for j in range(i+1,11):
        s.append((i,j))
print(s)

3重ループにすると、3つの組み合わせも列挙できます。

In [ ]:
s = []
for i in range(6):
    for j in range(i+1,6):
        for k in range(j+1,6):
            s.append((i,j,k))
print(s)
print(len(s))

関数にしてしまいましょう。

In [ ]:
def combination3(N):
    s = []
    for i in range(N):
        for j in range(i+1,N):
            for k in range(j+1,N):
                s.append((i,j,k))
    return s

print(combination3(5))


数字の範囲の代わりに、リストや集合を与える場合、繰り返しの範囲を書くのに困ってしまいます。ひとつの方法は次のような書きかたです。

In [ ]:
def combination3(L):
    s = []
    for i in range(len(L)):
        for j in range(i):
            for k in range(j):
                s.append((L[i],L[j],L[k]))
    return s

print(combination3(['A','B','C','D','E']))


イテレータを使うと、もっとシンプルに書けます。

In [ ]:
def combination3(L):
    for i in range(len(L)):
        for j in range(i):
            for k in range(j):
                yield L[i],L[j],L[k]

for x in combination3(['A','B','C','D','E']):
    print(x)


選びだす個数を3個に固定せず、自由に個数を選べるようにするには、再帰が必要です。

In [ ]:
def combinations(L,n):
    """
    Combine n elements out of list L.
    
    L: list of elements
    n: number of elements to be combined
    """
    if n == 0:
        yield []
    else:
        for i in range(len(L)):
            for x in combinations(L[i+1:], n-1):
                yield [L[i]] + x

for x in combinations(['A','B','C','D','E'],2):
    print(x)

結果を集合型にすると、要素の順番に意味がないことがより明確になります。

In [ ]:
def combinations(L,n):
    """
    Combine n elements out of list L.
    
    L: list of elements
    n: number of elements to be combined
    """
    if n == 0:
        yield set()
    else:
        for i in range(len(L)):
            for x in combinations(L[i+1:], n-1):
                yield set([L[i]]) | x

for x in combinations(['A','B','C','D','E'],2):
    print(x)

## 順列
与えられた要素を全部つかって、並べ方を列挙することを順列といいます。例えば、0と1の順列は01と10の2通りですが、一般に1〜Nの整数の順列の数はN!となり、かなりの個数になります。

これははじめから再帰を使って考えるのが良いでしょう。順列関数は、与えられたリストLの要素を並べかえたすべての順列を順に返すイテレータとして定義します。

    def permutations(L):
        ...
        yield *list_of_elements*


おおまかな手順は次の
1. 1〜Nから1つ数字を選び、iとする。
2. 残りの(N-1)の数字の順列はpermutations()で生成する。これをjとする。
3. iとjを連結したものをyieldする。


In [ ]:
def permutations(L):
    if len(L) == 0:
        yield []
    else:
        for k in range(len(L)):
            i = L[k]
            for j in permutations(L[:k]+L[k+1:]):
                yield [i]+j

for x in permutations([1,2,3]):
    print(x)

このようなプログラムは、なかなか間違いなく書くのは難しく、読むにもすこし経験が必要です。

そこで、標準ライブラリitertoolsの出番です。
## itertoolsの概要
itertoolsは、順列組み合わせの基本的なアルゴリズムを集めたものです。→https://docs.python.org/3/library/itertools.html

### 組みあわせ
5つの要素のなかから、2つを選ぶ組みあわせは、Fortranなどでは2重ループで書くのが一般的ですが、itertoolsを使うと非常に簡単に書けます。

In [ ]:
import itertools
for a,b in itertools.combinations([1,2,3,4,5], 2):
    print(a,b)

### 順列
5つの要素を並びかえるすべての順列に対して何かの処理をしたい場合も、繰り返しで書くのは骨がおれますが、itertoolsではとても簡単です。

In [ ]:
for p in itertools.permutations("ドレミソラ"):
    print(p)

### 重複順列
4x4x4の格子点を生成する問題は、重複順列とみなすことができます。itertoolsには重複順列ももちろんあります。

In [ ]:
for i in itertools.product([0,1,2,3],[0,1,2,3],[0,1,2,3]):
    print(i)

いくつも同じものを並べるのが面倒なので、繰り返し回数を指定します。

In [ ]:
for i in itertools.product(range(4),repeat=3):
    print(i)

## 使用例1

1次元のランダムウォークは、1歩ごとに右(+1)か左(-1)に進みます。10歩のランダムウォークの全経路は、

In [ ]:
for i in itertools.product([-1,+1],repeat=10):
    print(i)

和をとると、最終的な座標がわかります。

In [ ]:
for i in itertools.product([-1,+1],repeat=10):
    print(sum(i))

これを集計すると、2項分布になることがわかります。repeat=の部分を大きくするとなめらかになりますが、20を越えると暴走するかもしれません。

In [ ]:
%matplotlib inline
import pylab
positions = dict()
for i in itertools.product([-1,+1],repeat=10):
    print(i)
    pos = sum(i)
    if pos in positions:
        positions[pos] += 1
    else:
        positions[pos] = 1
#辞書のKeyをソートしたもの
x = sorted(positions)
#対応する累積数
y = [positions[pos] for pos in x]

pylab.plot(x,y)

## 使用例2
にせアンパンマンをたくさん作ります。join関数は、文字列のリストをひとつの文字列に合体させます。また、random.shuffle関数は、リストの順序をランダムにいれかえます。

応用として、「いつだれがどこでなにをどうした」ゲームをコンピュータに実行させるという遊びかたもあります。

In [ ]:
import itertools
import random
にせあんぱんまん = []
for あんぱんまん in itertools.product("アマヌ","リソンシツ","パバベペ","リソンシツ","アマヌ","リソンシツ"):
    にせあんぱんまん.append("".join(あんぱんまん))
random.shuffle(にせあんぱんまん)
print(にせあんぱんまん)

## 練習問題
### 問題1
以前どこかで説明をしたような気がしますが、eval(文字列)で、文字列をpythonプログラムとして評価できます。

たとえば、以下の例では、文字列"10\*20\*3"を評価して、数値63が得られます。

In [ ]:
x = eval("10*20+3")
x

これを使うと、プログラムを実行している最中に、変化する数式の値を計算できます。

人間が入力した数式をその場で数値に直せるので、超高性能関数電卓が3行で作れます。

In [ ]:
from math import *
while True:
    print(eval(input("Formula:")))

さて、1〜5の数字の間に、四則演算子(\*, /, +, -)をはさみこんで、答が20になるような数式をすべて表示するプログラムを作って下さい。数字の順番を変えてはいけません。たとえば次のような式です。

In [ ]:
1+2+3*4+5

それができたら、改造して、1〜9の数字の間に、四則演算子(\*, /, +, -)をはさみこんで、答が100になるような数式をすべて表示するプログラムを作って下さい。

その中で、いちばん演算子が少なくて済む(=式の文字数が少ない)のはどんな式でしょうか。

この手の問題は、プログラムを書ける人にとっては楽勝ですが、そうでない人はまず正解を得ることができません。そして、現実の問題は、おそらくプログラムなしには解けない問題のほうが多いはずです。
### 問題2
正の整数のリストを与えられたとき、数を並び替えて可能な最大数を返す関数を記述せよ。例えば、[50, 2, 1, 9]が与えられた時、95021が答えとなる。(https://blog.svpino.com/2015/05/07/five-programming-problems-every-software-engineer-should-be-able-to-solve-in-less-than-1-hour より)

### 問題3
覆面算HAWAII+IDAHO+IOWA+OHIO==STATESを解け。ただし、それぞれのアルファベットは0〜9の異なる数字が入り、一番上の桁は0でない。